# Steam Games: Data Cleaning Notebook

**Purpose:** Prepare a clean, analysis‑ready subset of the Steam Games dataset.

**Outputs:** `data/steam_games_cleaned.csv`



## 1. Setup & Config

- Set the path to raw dataset (CSV).  
- If file name differs, update `RAW_DATA_PATH`.  

In [ ]:
import kagglehub
# Downloading the dataset
# path = kagglehub.dataset_download("fronkongames/steam-games-dataset")
# print("Path to dataset files:", path)

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

# Configure paths 
CLEAN_PATH = Path("data/steam_games_cleaned.csv") # output file
FIXED_DATA_PATH = Path("data/games_fixed.csv") # <-- replace with your actual filename
CLEAN_PATH, FIXED_DATA_PATH

(PosixPath('data/steam_games_cleaned.csv'), PosixPath('data/games_fixed.csv'))


## 2. Load & Inspect


In [8]:
# Load the dataset 
if not FIXED_DATA_PATH.exists():
    raise FileNotFoundError(f"Raw dataset not found at {FIXED_DATA_PATH}")

df_raw = pd.read_csv(FIXED_DATA_PATH)
print(df_raw.shape)
df_raw.head(3)

(111452, 40)


,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,Discount,DLC count,About the game,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,0,Galactic Bowling is an exaggerated and stylize...,...,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,0,THE LAW!! Looks to be a showdown atop a train....,...,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,0,Jolt Project: The army now has a new robotics ...,...,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [9]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111452 entries, 0 to 111451
Data columns (total 40 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   AppID                       111452 non-null  int64  
 1   Name                        111446 non-null  object 
 2   Release date                111452 non-null  object 
 3   Estimated owners            111452 non-null  object 
 4   Peak CCU                    111452 non-null  int64  
 5   Required age                111452 non-null  int64  
 6   Price                       111452 non-null  float64
 7   Discount                    111452 non-null  int64  
 8   DLC count                   111452 non-null  int64  
 9   About the game              104969 non-null  object 
 10  Supported languages         111452 non-null  object 
 11  Full audio languages        111452 non-null  object 
 12  Reviews                     10624 non-null   object 
 13  Header image  

In [10]:
#before cleaning 
df_raw.describe()

,AppID,Peak CCU,Required age,Price,Discount,DLC count,Metacritic score,User score,Positive,Negative,Score rank,Achievements,Recommendations,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks
count,1.114520e+05,1.114520e+05,111452.000000,111452.000000,111452.000000,111452.000000,111452.000000,111452.000000,1.114520e+05,111452.000000,44.000000,111452.000000,1.114520e+05,111452.000000,111452.000000,111452.000000,111452.000000
mean,1.716972e+06,1.777215e+02,0.254208,7.061568,0.464209,0.449530,2.623354,0.030408,7.543525e+02,125.859177,98.909091,17.511144,6.163715e+02,81.247290,9.174954,72.651330,9.891038
std,9.203859e+05,8.390462e+03,2.035653,12.563246,3.503658,12.006677,13.736245,1.565136,2.139410e+04,4002.844431,0.857747,150.139008,1.573854e+04,999.935906,168.201030,1321.333137,183.232812
min,1.000000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,97.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,9.362550e+05,0.000000e+00,0.000000,0.990000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,98.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
50%,1.665065e+06,0.000000e+00,0.000000,3.990000,0.000000,0.000000,0.000000,0.000000,3.000000e+00,1.000000,99.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
75%,2.453585e+06,1.000000e+00,0.000000,9.990000,0.000000,0.000000,0.000000,0.000000,2.900000e+01,8.000000,100.000000,17.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
max,3.671840e+06,1.311366e+06,21.000000,999.980000,92.000000,2366.000000,97.000000,100.000000,5.764420e+06,895978.000000,100.000000,9821.000000,3.441592e+06,145727.000000,19159.000000,208473.000000,19159.000000


In [11]:
df_raw.nunique() #check unique values in each column

AppID                         111452
Name                          110325
Release date                    4939
Estimated owners                  14
Peak CCU                        1760
Required age                      19
Price                            756
Discount                          43
DLC count                         98
About the game                104404
Supported languages            14861
Full audio languages            2862
Reviews                        10480
Header image                  111342
Website                        37660
Support url                    33236
Support email                  54954
Windows                            2
Mac                                2
Linux                              2
Metacritic score                  74
Metacritic url                  3905
User score                        33
Positive                        4638
Negative                        2354
Score rank                         4
Achievements                     436
R

In [12]:
print(df_raw.columns.tolist())

df_raw[["AppID","Name","Release date","Estimated owners","Peak CCU","Required age","Price","Discount","DLC count","About the game"]].head(3)

['AppID', 'Name', 'Release date', 'Estimated owners', 'Peak CCU', 'Required age', 'Price', 'Discount', 'DLC count', 'About the game', 'Supported languages', 'Full audio languages', 'Reviews', 'Header image', 'Website', 'Support url', 'Support email', 'Windows', 'Mac', 'Linux', 'Metacritic score', 'Metacritic url', 'User score', 'Positive', 'Negative', 'Score rank', 'Achievements', 'Recommendations', 'Notes', 'Average playtime forever', 'Average playtime two weeks', 'Median playtime forever', 'Median playtime two weeks', 'Developers', 'Publishers', 'Categories', 'Genres', 'Tags', 'Screenshots', 'Movies']


,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,Discount,DLC count,About the game
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,0,Galactic Bowling is an exaggerated and stylize...
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,0,THE LAW!! Looks to be a showdown atop a train....
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,0,Jolt Project: The army now has a new robotics ...


# Drop NLP specific/ unwanted columns
    "Header image", "Website", "Support url", "Support email",
    "Metacritic url", "Screenshots", "Movies",
    "Score rank", "Reviews", "Notes", "Supported languages ", " Full audio languages"

- Change numerical nans to 0
- Change categorical nans to 'Unknown'
- Drop duplicate AppIDs

In [13]:
# Change numerical nans to 0
# Change categorical nans to 'Unknown'
# Drop duplicate AppIDs


## Drop irrelevant / sparse columns
drop_cols = [ "Supported languages", "Full audio languages" ,"Reviews", "Release date", 
    "Header image", "Website", "Support url", "Support email",
    "Metacritic url", "Notes","Developers ", "Screenshots", "Movies", "Score rank" #NLP specific
]

# drop_cols =[]

df = df_raw.copy()

# Parse release year from release date
df["Release year"] = pd.to_datetime(df["Release date"], errors="coerce", infer_datetime_format=True).dt.year

df.update(df)

df = df.drop(columns=drop_cols, errors="ignore")

# Handle nulls
# Drop rows missing critical identifiers
df = df.dropna(subset=["AppID", "Name", "Release year", "Price"])

df['Release year']= df["Release year"].astype(int)

# Fill numeric NaNs with 0 
num_fill_zero = ["Achievements", "Recommendations", 
                 "Average playtime forever", "Median playtime forever"]
for col in num_fill_zero:
    if col in df.columns:
        df[col] = df[col].fillna(0)

# Fill categorical NaNs with 'Unknown'
cat_fill_unknown = ["Genres", "Categories", "Tags", "Developers", "Publishers"]
for col in cat_fill_unknown:
    if col in df.columns:
        df[col] = df[col].fillna("Unknown")

df = df.drop_duplicates(subset=["AppID"])
df.info()

/tmp/ipykernel_15513/1203413935.py:17: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["Release year"] = pd.to_datetime(df["Release date"], errors="coerce", infer_datetime_format=True).dt.year


<class 'pandas.core.frame.DataFrame'>
Index: 111315 entries, 0 to 111451
Data columns (total 28 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   AppID                       111315 non-null  int64  
 1   Name                        111315 non-null  object 
 2   Estimated owners            111315 non-null  object 
 3   Peak CCU                    111315 non-null  int64  
 4   Required age                111315 non-null  int64  
 5   Price                       111315 non-null  float64
 6   Discount                    111315 non-null  int64  
 7   DLC count                   111315 non-null  int64  
 8   About the game              104838 non-null  object 
 9   Windows                     111315 non-null  bool   
 10  Mac                         111315 non-null  bool   
 11  Linux                       111315 non-null  bool   
 12  Metacritic score            111315 non-null  int64  
 13  User score         

In [14]:
missing_values_count = df_raw.isnull().sum()
missing_values_count[0:40]  # Display counts of missing values for the first 40 columns

AppID                              0
Name                               6
Release date                       0
Estimated owners                   0
Peak CCU                           0
Required age                       0
Price                              0
Discount                           0
DLC count                          0
About the game                  6483
Supported languages                0
Full audio languages               0
Reviews                       100828
Header image                       0
Website                        64994
Support url                    60693
Support email                  19025
Windows                            0
Mac                                0
Linux                              0
Metacritic score                   0
Metacritic url                107447
User score                         0
Positive                           0
Negative                           0
Score rank                    111408
Achievements                       0
R

In [15]:
total_cells = np.prod(df_raw.shape)
print(total_cells)
total_missing = missing_values_count.sum()
total_missing
# percent of data that is missing
percent_missing = (total_missing/total_cells) * 100
print(percent_missing)

4458080
14.40936456950077


In [16]:
df_copy = df.copy()
missing_values_count1 = df_copy.isnull().sum()
missing_values_count1[0:40]  # Display counts of missing values for the first 40 columns

AppID                            0
Name                             0
Estimated owners                 0
Peak CCU                         0
Required age                     0
Price                            0
Discount                         0
DLC count                        0
About the game                6477
Windows                          0
Mac                              0
Linux                            0
Metacritic score                 0
User score                       0
Positive                         0
Negative                         0
Achievements                     0
Recommendations                  0
Average playtime forever         0
Average playtime two weeks       0
Median playtime forever          0
Median playtime two weeks        0
Developers                       0
Publishers                       0
Categories                       0
Genres                           0
Tags                             0
Release year                     0
dtype: int64

In [17]:

total_cells1 = np.prod(df_copy.shape)
print(total_cells1)
total_missing1 = missing_values_count1.sum()
# percent of data that is missing
percent_missing = (total_missing1/total_cells1) * 100
print(percent_missing)

3116820
0.20780795811115174


In [18]:
#save cleaned data
df.to_csv(CLEAN_PATH, index=False)
print(f"Cleaned data saved to {CLEAN_PATH}")

Cleaned data saved to data/steam_games_cleaned.csv


In [19]:
#print first 100 rows of appid
print(df.iloc[100:210])
df['Name'].head(100)

       AppID                                               Name  \
101   863490                                     Bighead Runner   
102   396640                                         TowerClimb   
103   726020                                 Mission Ammunition   
104   424060                                           The Glow   
105  1943590                                     溪风谷之战 Playtest   
..       ...                                                ...   
207  2010700                                   Hunter Survivors   
208  1089830  Monster Energy Supercross - The Official Video...   
209   403070                      Silver Creek Falls: Chapter 2   
210   811440                                Gripper's Adventure   
211   491010                        IN-VERT: Definitive Edition   

    Estimated owners  Peak CCU  Required age  Price  Discount  DLC count  \
101    20000 - 50000         0             0   0.99         0          1   
102        0 - 20000         0             

0                            Galactic Bowling
1                                Train Bandit
2                                Jolt Project
3                                    Henosis™
4                       Two Weeks in Painland
                        ...                  
96                        Oxygen Not Included
97               Laruaville 10 Match 3 Puzzle
98              Slot Shots Pinball Collection
99     Laid-Back Camp - Virtual - Lake Motosu
100                                       Kor
Name: Name, Length: 100, dtype: object

In [20]:
game_opinion= df[["Name",'Release year', 'Estimated owners', 'Required age','Price', "Recommendations","User score","Positive","Negative","Achievements"]]
game_opinion.head(200)

,Name,Release year,Estimated owners,Required age,Price,Recommendations,User score,Positive,Negative,Achievements
0,Galactic Bowling,2008,0 - 20000,0,19.99,0,0,6,11,30
1,Train Bandit,2017,0 - 20000,0,0.99,0,0,53,5,12
2,Jolt Project,2021,0 - 20000,0,4.99,0,0,0,0,0
3,Henosis™,2020,0 - 20000,0,5.99,0,0,3,0,0
4,Two Weeks in Painland,2020,0 - 20000,0,0.00,0,0,50,8,17
...,...,...,...,...,...,...,...,...,...,...
197,Storage Chase,2021,0 - 20000,0,0.99,0,0,0,0,24
198,Ghosts of War: Battle Royale WW2 Shooting games,2021,0 - 0,0,0.00,0,0,0,0,0
199,Space Tycoon | 星际大亨,2019,0 - 20000,0,16.99,0,0,7,8,0
200,Fables of Talumos,2019,0 - 20000,0,6.99,0,0,8,0,16


In [21]:
# Check ranges and summary stats of numeric columns
num_cols = df.select_dtypes(include=[np.number]).columns
summary = df[num_cols].describe().T
summary["num_missing"] = df[num_cols].isna().sum()
summary["num_negatives"] = (df[num_cols] < 0).sum()
summary["num_zeros"] = (df[num_cols] == 0).sum()
summary

,count,mean,std,min,25%,50%,75%,max,num_missing,num_negatives,num_zeros
AppID,111315.0,1.718009e+06,920274.441162,10.0,937315.00,1666720.00,2454595.00,3671840.00,0,0,0
Peak CCU,111315.0,1.771411e+02,8391.712319,0.0,0.00,0.00,1.00,1311366.00,0,0,82867
Required age,111315.0,2.540358e-01,2.035141,0.0,0.00,0.00,0.00,21.00,0,0,109572
Price,111315.0,7.060579e+00,12.563615,0.0,0.99,3.99,9.99,999.98,0,0,23199
Discount,111315.0,4.647801e-01,3.505776,0.0,0.00,0.00,0.00,92.00,0,0,108824
DLC count,111315.0,4.494453e-01,12.013856,0.0,0.00,0.00,0.00,2366.00,0,0,97553
Metacritic score,111315.0,2.616045e+00,13.717813,0.0,0.00,0.00,0.00,97.00,0,0,107326
User score,111315.0,3.044513e-02,1.566098,0.0,0.00,0.00,0.00,100.00,0,0,111271
Positive,111315.0,7.489144e+02,21349.039345,0.0,0.00,3.00,29.00,5764420.00,0,0,40194
Negative,111315.0,1.255722e+02,4004.970252,0.0,0.00,1.00,8.00,895978.00,0,0,55098


In [22]:
print(df.shape)
df[["AppID","Name","Release year","Price"]].head()
df.isna().sum().sort_values(ascending=False).head(10)

(111315, 28)


About the game      6477
AppID                  0
Estimated owners       0
Name                   0
Peak CCU               0
Required age           0
Discount               0
Price                  0
DLC count              0
Windows                0
dtype: int64

In [23]:
# display(df_raw.head(3))
# Candidate keys
for key in [["AppID"], ["Name"], ["Publishers"], ["Developers"]]:
    if all(c in df.columns for c in key):
        dup = df.duplicated(subset=key).sum()
        print(f"Duplicates on {key}: {dup}")

Duplicates on ['AppID']: 0
Duplicates on ['Name']: 1119
Duplicates on ['Publishers']: 54352
Duplicates on ['Developers']: 46725



## 3. Notes for README.md

- **Source:** Steam games dataset (Kaggle/Steam API).   
- **Cleaning decisions:**   
  - Parsed `Release year` from `release_date` (dropped rows missing critical fields).  
  - Removed duplicates by `AppID` and the NaN values  
- **Output:** `data/steam_games_cleaned.csv`  
- EDA-ready columns include `Price`, `Positive`, `Negative`, `Genres`, `Release year`.
